In [0]:
%sql
-- Create a view for pairs of entitlements
create or replace view access_pairs as
select distinct
  identityId
  ,case when e1 > e2 then e2 else e1 end as e1
  ,case when e1 < e2 then e2 else e1 end as e2
  from (
    select e1.identityId, e1.id as e1, e1.name, e2.id as e2, e2.name
    -- Filter on entitlements, the most recent day version, and remove birthright items
    from
      (select identityId, id, name from dunker_databricks_space.sailpoint.access_assignments WHERE day = (SELECT MAX(day) FROM dunker_databricks_space.sailpoint.access_assignments) AND type = 'ENTITLEMENT' AND name not like 'Birthright-%' AND name != 'All_Users') e1
    inner join
      (select identityId, id, name from dunker_databricks_space.sailpoint.access_assignments WHERE day = (SELECT MAX(day) FROM dunker_databricks_space.sailpoint.access_assignments) AND type = 'ENTITLEMENT' AND name not like 'Birthright-%' AND name != 'All_Users') e2
    on e1.identityId = e2.identityId
    and e1.id != e2.id
  )

In [0]:
%sql
-- Create a vew that represents the nodes
create or replace view nodes as
select id, name, count(1) as `size`
from sailpoint.access_assignments
WHERE day = (SELECT MAX(day) FROM dunker_databricks_space.sailpoint.access_assignments)
AND type = 'ENTITLEMENT' AND name not like 'Birthright-%' AND name != 'All_Users'
group by id, name
having count(1) > 10

In [0]:
%python
import json
df = spark.sql('select id, name, size from nodes')
nodes = df.toJSON().map(lambda j: json.loads(j)).collect()
#print(nodes)

In [0]:
%sql
create or replace view edges as
select
  e1 as `source`
  ,e2 as `target`
  ,count(1) as `weight`
from access_pairs
where e1 in (select id from nodes)
and e2 in (select id from nodes)
group by e1, e2
order by count(1) desc

In [0]:
%python
import json
df = spark.sql('select source, target, weight from edges')
edges = df.toJSON().map(lambda j: json.loads(j)).collect()
#print(edges)

In [0]:
%python
html = """
  <html>
  <head>
  <meta charset="utf-8" />
  <script src="https://d3js.org/d3-force.v1.min.js"></script>
  <script src="https://d3js.org/d3.v4.min.js"></script>

</head>
  <body>

    <div id="graphDiv"></div>

<hr/>

    <button onclick="download('png')">
      Download PNG
    </button>

    <button onclick="download('jpg')">
      Download JPG
    </button>


 <script>
      var data = { nodes: %s, links: %s };
   
   
      // Get the maximum weight for an edge
      var maxWeight = 0;
      for (var i = 0; i < data.links.length; i++) {
        if (maxWeight < data.links[i].weight) maxWeight = data.links[i].weight;
      }

      var height = 1000;
      var width = 2000;
      

      // Append the canvas to the HTML document
      var graphCanvas = d3
        .select("#graphDiv")
        .append("canvas")
        .attr("width", width + "px")
        .attr("height", height + "px")
        .node();

      var context = graphCanvas.getContext("2d");

      var div = d3
        .select("body")
        .append("div")
        .attr("class", "tooltip")
        .style("opacity", 0);

      var simulation = d3
        .forceSimulation()
        .force(
          "link",
          d3
            .forceLink()
            .id(function(d) {
              return d.id;
            })
            .distance(function(d) {
              return 15 * (maxWeight - d.weight);
            })
            .strength(function(d) {
              return 1 * (d.weight / maxWeight);;
            })
        )
        .force("charge", d3.forceManyBody().strength(-750))
        .force("center", d3.forceCenter(width / 2, height / 2))
        .force("x", d3.forceX(width / 2).strength(0.01))
        .force("y", d3.forceY(height / 2).strength(0.01))
        .alphaTarget(0)
        .alphaDecay(0.05);

      var transform = d3.zoomIdentity;

      initGraph(data);

      function initGraph(tempData) {
        function zoomed() {
          console.log("zooming");
          transform = d3.event.transform;
          simulationUpdate();
        }

        d3.select(graphCanvas)
          .call(
            d3
              .drag()
              .subject(dragsubject)
              .on("start", dragstarted)
              .on("drag", dragged)
              .on("end", dragended)
          )
          .call(
            d3
              .zoom()
              .scaleExtent([1 / 10, 8])
              .on("zoom", zoomed)
          );

        function dragsubject() {
          var i,
            x = transform.invertX(d3.event.x),
            y = transform.invertY(d3.event.y),
            dx,
            dy;
          for (i = tempData.nodes.length - 1; i >= 0; --i) {
            node = tempData.nodes[i];
            dx = x - node.x;
            dy = y - node.y;

            let radius = Math.min(30, node.size)
            if (dx * dx + dy * dy < radius * radius) {
              node.x = transform.applyX(node.x);
              node.y = transform.applyY(node.y);

              return node;
            }
          }
        }

        function dragstarted() {
          if (!d3.event.active) simulation.alphaTarget(0.3).restart();
          d3.event.subject.fx = transform.invertX(d3.event.x);
          d3.event.subject.fy = transform.invertY(d3.event.y);
        }

        function dragged() {
          d3.event.subject.fx = transform.invertX(d3.event.x);
          d3.event.subject.fy = transform.invertY(d3.event.y);
        }

        function dragended() {
          if (!d3.event.active) simulation.alphaTarget(0);
          d3.event.subject.fx = null;
          d3.event.subject.fy = null;
        }

        simulation.nodes(tempData.nodes).on("tick", simulationUpdate);

        simulation.force("link").links(tempData.links);

        function render() {}

        function simulationUpdate() {
          context.save();

          context.clearRect(0, 0, width, height);
          context.translate(transform.x, transform.y);
          context.scale(transform.k, transform.k);

          // Draw the links
          tempData.links.forEach(function(d) {
            context.beginPath();
            context.lineWidth = Math.min(d.weight, 40);
            context.strokeStyle = "rgba(0, 158, 227, .3)";
            context.moveTo(d.source.x, d.source.y);
            context.lineTo(d.target.x, d.target.y);
            context.stroke();
          });

          // Draw the nodes
          tempData.nodes.forEach(function(d, i) {
            context.beginPath();
            context.arc(d.x, d.y, Math.min(30, d.size), 0, 2 * Math.PI, true);
            context.fillStyle = "rgba(0, 158, 227, 0.8)";
            context.fill();
            context.fillStyle = "rgba(0, 0, 0, 1)";
            context.fillText(d.name, d.x + 10, d.y);
          });

          context.restore();
          
        }
      }
      
    function download(type) {
        var canvas = document.querySelector("canvas");

        var imgUrl;
        
        if (type === "png") 
          imgUrl = canvas.toDataURL("image/png");
        else if (type === "jpg") 
          imgUrl = canvas.toDataURL("image/png");

        window.open().document.write('<img src="' + imgUrl + '" />');
      }
    </script>
    
  </body>
</html>
""" % (nodes, edges)

#print(html)
displayHTML(html)